In [201]:
import os
import sys
import shutil
import pandas
import numpy
import ipystata

from ipystata.config import config_stata
config_stata('/home/kmongeon/research/stata14/stata-mp',force_batch=True)

pandas.options.display.max_columns = 50


In [202]:
shutil.rmtree('out')
os.mkdir('out')


(70 vars, 427 obs)
(note: file ./out/di.dta not found)
file ./out/di.dta saved
(14 vars, 421 obs)
(40 observations deleted)(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(note: file ./out/dt.dta not found)
file ./out/dt.dta saved
. * Nota: July  2016(32 vars, 109 obs)
(59 observations deleted)
(note: file ./out/db1.dta not found)
file ./out/db1.dta saved
. import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
(34 vars, 473 obs)
(460 observations deleted)
(note: file ./out/db2.dta not found)
file ./out/db2.dta saved
. use ./out/di, clear

    Result                           # of obs.
    -----------------------------------------
    not matched                            58
        from master                        52  
        from using                          6  

    matched                               375  
    -----------------------------------------

    Result                           #

In [204]:
dm.describe()

,id,session,sequence,seq,nota,falk,rsos,tsos,grip,ptiso,pt60,godin,paq,ntxc,cint,calc,vitd,vitd2,mvh,matu,age,gender,griph,ptisoh,rsosh,tsosh
count,425.000000,425.000000,425.000000,425.000000,50.0,11.0,415.000000,415.000000,379.000000,310.000000,310.000000,422.000000,425.000000,385.000000,422.000000,421.000000,421.000000,421.000000,307.000000,425.000000,425.000000,425.000000,425.000000,422.000000,425.000000,422.000000
mean,326.197647,1.430588,4.197647,1.868235,1.0,1.0,3815.551807,3687.079518,23.438074,124.041290,84.050201,72.606635,3.144819,540.346619,1598.184814,976.547974,4.703625,189.981476,106.180466,-0.893577,11.850613,0.468235,23.412334,125.861549,3815.268311,3686.898193
std,197.549787,0.495742,1.833518,0.877686,0.0,0.0,100.909471,109.038357,7.364498,53.311306,33.349968,39.683328,0.662686,265.354980,482.215729,455.661133,3.152561,127.345634,41.705219,1.942006,2.003003,0.499578,6.997608,46.829910,60.782661,59.051323
min,100.000000,1.000000,1.000000,1.000000,1.0,1.0,3542.000000,3367.000000,9.000000,32.647167,28.224886,0.000000,1.250000,56.308327,469.496002,51.714001,0.000000,0.000000,13.570000,-5.300000,7.500000,0.000000,9.000000,32.647167,3554.000000,3367.000000
25%,138.000000,1.000000,3.000000,1.000000,1.0,1.0,3746.500000,3619.000000,18.000000,84.335386,58.508888,44.000000,2.718600,369.744904,1263.682983,652.146973,2.649000,107.401001,78.139999,-2.290000,10.340000,0.000000,18.500000,92.210487,3780.298096,3647.084717
50%,189.000000,1.000000,4.000000,2.000000,1.0,1.0,3820.000000,3689.000000,22.250000,114.025639,76.993565,65.000000,3.197700,514.754761,1548.725037,915.741028,4.204000,173.792007,99.110001,-1.000000,11.850000,0.000000,22.500000,120.712017,3814.430908,3683.248779
75%,530.000000,2.000000,6.000000,2.000000,1.0,1.0,3878.000000,3758.000000,27.500000,156.607567,102.830297,92.750000,3.663800,662.435608,1878.681793,1196.994019,6.464000,263.270996,133.389999,0.450000,13.230000,1.000000,27.000000,150.717136,3853.525146,3724.732605
max,583.000000,2.000000,7.000000,4.000000,1.0,1.0,4140.000000,4015.000000,53.000000,343.836792,237.822296,234.000000,4.610400,1754.324463,3624.724121,2518.876953,21.211000,848.426025,235.000000,4.950000,17.549999,1.000000,53.000000,343.836792,3971.000000,3962.000000


In [309]:
%%stata --output dm  --noprint

import delimited ./data/IzzyLongitudinalWorkingMarch2015.csv,  case(lower) clear 
order id session sequence
sort id sequence
save ./out/di, replace

import delimited ./data/torque.csv, case(lower) clear
drop if session==0
drop sequence
rename new_sequence sequence
order id session sequence
sort id  sequence
foreach var in ptiso pt60 pt240 {
replace `var' = . if `var'==-9999
}
save ./out/dt, replace

** Identify potentially data with errors
* Nota: July  2016
import delimited ./data/NegativeIDs.csv, case(lower) clear
keep nota id sequence 
keep if nota==1
save ./out/db1, replace

* Baraket: Nov 2016
import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
keep falk id sequence
keep if falk==1
save ./out/db2, replace

* merge
use ./out/di, clear
sort id sequence
merge 1:1 id sequence using ./out/dt,  nogen
merge 1:1 id sequence using ./out/db1, nogen
merge 1:1 id sequence using ./out/db2, nogen

rename radius_sos rsos
rename tibial_sos tsos

rename grip_strength_best grip 
rename mat_offset_new matu

rename godin_pa godin
rename ntxcreat ntxc
rename caloricintake cint
renam calcium calc 
rename vitd vitd 
rename v48 vitd2
rename paq_total_score paq
rename totmvh mvh

global U id session sequence 
global Y rsos tsos
global X grip ptiso pt60 godin paq ntxc cint calc vitd vitd2 mvh
global Z matu age gender
global D nota falk

keep $U $Y $X $Z $D

***grip only

*drop if missing(rsos)
*drop if missing(tsos)
*drop if missing(grip)
*drop if missing(ptiso) 
drop if matu>5
drop if id==517 & sequence==7

sort id sequence
by id: egen seq = seq()
global U id session sequence seq

order $U $D $Y $X $Z 

xtset id seq
count 

capture drop griph 
reg grip rsos 
predict griph 
replace griph = grip if (!missing(grip) & nota!=1)

reg ptiso matu

capture drop ptisoh  
reg ptiso tsos
predict ptisoh 
replace ptisoh = ptiso if !missing(ptiso) 

capture drop ptisoh2  
reg ptisoh matu
predict ptisoh2 
replace ptisoh = ptisoh2 if missing(ptisoh) 
drop ptisoh2

capture drop rsosh  
reg rsos griph  
predict rsosh 
replace rsosh = rsos if (!missing(rsos) & nota!=1)

capture drop tsosh  
reg tsos ptisoh  
predict tsosh 
replace tsosh = tsos if (falk!=1 & !missing(tsos))

capture drop mvhh  
reg mvh ptisoh 
predict mvhh 
replace mvhh = mvh if !missing(mvh)

capture drop ntxch  
reg ntxc rsosh 
predict ntxh 
replace ntxh = ntxc if !missing(ntxc)

capture drop godinh  
reg godin tsosh
predict godinh 
replace godinh = godin if !missing(godin)

capture drop paqh  
reg paq matu 
predict paqh 
replace paqh = paq if !missing(paq)

capture drop calch  
reg calc matu
predict calch 
replace calch = calc if !missing(calc)

capture drop cinth  
reg cint matu
predict cinth 
replace cinth = cint if !missing(cint)

capture drop vitdh  
reg vitd matu
predict vitdh 
replace vitdh = vitd if !missing(vitd)

sort id seq
by id: egen trips = max(seq)
order trips, after(seq)



In [310]:
dm['rsoshs'] = (dm['rsosh'] - dm['rsosh'].mean()) / dm['rsosh'].std()
dm['tsoshs'] = (dm['tsosh'] - dm['tsosh'].mean()) / dm['tsosh'].std()
dm['griphs'] = (dm['griph'] - dm['griph'].mean()) / dm['griph'].std()
dm['ptisohs'] = (dm['ptisoh'] - dm['ptisoh'].mean()) / dm['ptisoh'].std()
dm['ntxhs'] = (dm['ntxh'] - dm['ntxh'].mean()) / dm['ntxh'].std()
dm['mvhhs'] = (dm['mvhh'] - dm['mvhh'].mean()) / dm['mvhh'].std()
dm['godinhs'] = (dm['godinh'] - dm['godinh'].mean()) / dm['godinh'].std()
dm['cinths'] = (dm['cinth'] - dm['cinth'].mean()) / dm['cinth'].std()
dm['vitdhs'] = (dm['vitdh'] - dm['vitdh'].mean()) / dm['vitdh'].std()

In [318]:
%%stata --data dm --openstata 


/*
gsem ///
(rsosh <-  griph  matu ntxh       M1[id]) ///
(tsosh <-  ptisoh matu ntxh mvhh  M2[id]) ///
(griph <-   matu godinh cinth vitdh) ///
(ptisoh <-  matu godinh cinth vitdh) , constraints(1)
matrix b = e(b)
*/

gsem ///
(rsoshs <-  griphs  matu ntxhs       M1[id]) ///
(tsoshs <-  ptisohs matu ntxhs  mvhhs M1[id]) ///
(griphs <-   matu godinhs   ) ///
(ptisohs <-  matu godinhs   )
matrix b = e(b)



gsem ///
(rsoshs <-  griphs  matu ntxhs        M1[id]) ///
(tsoshs <-  ptisohs matu ntxhs  mvhhs M2[id]) ///
(griphs <-   matu godinhs   M3[id] ) ///
(ptisohs <-  matu godinhs   M4[id] )
matrix b = e(b)


'Failed to open Stata'

In [322]:
%%stata --data dm --openstata 

gsem ///
(rsoshs <-  griphs  matu ntxhs       M1[id]) 


> (rsoshs <-  griphs  matu ntxhs       M1[id]) 

Fitting fixed-effects model:

Iteration 0:   log likelihood = -527.40929  
Iteration 1:   log likelihood = -527.40929  

Refining starting values:

Grid node 0:   log likelihood = -517.75534

Fitting full model:

Iteration 0:   log likelihood = -517.75534  (not concave)
Iteration 1:   log likelihood = -482.79777  
Iteration 2:   log likelihood =    -477.48  
Iteration 3:   log likelihood = -477.06993  
Iteration 4:   log likelihood = -477.06912  
Iteration 5:   log likelihood = -477.06912  

Generalized structural equation model           Number of obs     =        425
Response       : rsoshs
Family         : Gaussian
Link           : identity
Log likelihood = -477.06912

 ( 1)  [rsoshs]M1[id] = 1
------------------------------------------------------------------------------
             |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
rsoshs

In [324]:
%%stata --data dm --openstata 


set seed 12345

bayesmh rsoshs griphs  matu ntxhs, reffects(id) likelihood(normal({var_0})) noconstant  ///
prior({rsoshs:i.id},  normal({rsoshs:_cons},{var_id})) ///
prior({rsoshs:_cons}, normal(0, 100)) ///
prior({rsoshs:griphs}, normal(0, 100)) ///
prior({rsoshs:matu}, normal(0, 100)) ///
prior({rsoshs:ntxhs}, normal(0, 100)) ///
prior({var_0},        igamma(0.001, 0.001)) ///
prior({var_id},       igamma(0.001, 0.001)) ///
block({rsoshs:_cons}, gibbs)  ///
block({var_0}, gibbs) block({var_id}, gibbs) ///
mcmcsize(5000) dots



> prior({rsoshs:i.id},  normal({rsoshs:_cons},{var_id})) ///
> prior({rsoshs:_cons}, normal(0, 100)) ///
> prior({rsoshs:griphs}, normal(0, 100)) ///
> prior({rsoshs:matu}, normal(0, 100)) ///
> prior({rsoshs:ntxhs}, normal(0, 100)) ///
> prior({var_0},        igamma(0.001, 0.001)) ///
> prior({var_id},       igamma(0.001, 0.001)) ///
> block({rsoshs:_cons}, gibbs)  ///
> block({var_0}, gibbs) block({var_id}, gibbs) ///
> mcmcsize(5000) dots
  
Burn-in 2500 aaaaaaaaa1000aaaaaaaaa2000aaaaa done
Simulation 5000 .........1000.........2000.........3000.........4000.........5000 done

Model summary
------------------------------------------------------------------------------
Likelihood: 
  rsoshs ~ normal(xb_rsoshs,{var_0})

Priors: 
               {rsoshs:i.id} ~ normal({rsoshs:_cons},{var_id})             (1)
  {rsoshs:griphs matu ntxhs} ~ normal(0,100)                               (1)
                     {var_0} ~ igamma(0.001,0.001)
              {rsoshs:_cons} ~ normal(0,100)

Hype

In [341]:
%%stata --data dm --openstata 

gsem ///
(rsoshs <-  griphs  matu ntxhs       M1[id]) ///
(griphs <- matu)


set seed 12345

bayesmh ///
(rsoshs griphs, reffects(id) noconstant likelihood(normal({var_0})) ) ///
(griphs matu, reffects(id) noconstant likelihood(normal({var_1})) ), ///
prior({rsoshs:i.id},  normal({rsoshs:_cons},{var_id})) ///
prior({rsoshs:_cons}, normal(0, 100)) ///
prior({rsoshs:griphs}, normal(0, 100)) ///
///
prior({griphs:i.id},  normal({griphs:_cons},{var_id2})) ///
prior({griphs:_cons}, normal(0, 100)) ///
prior({griphs:matu}, normal(0, 100)) ///
///
prior({var_0},        igamma(0.001, 0.001)) ///
prior({var_1},        igamma(0.001, 0.001)) ///
prior({var_id},       igamma(0.001, 0.001)) ///
prior({var_id2},       igamma(0.001, 0.001)) ///
block({rsoshs:_cons}, gibbs) ///
block({var_0}, gibbs) ///
block({var_id}, gibbs)  ///
///
block({griphs:_cons}, gibbs) ///
block({var_1}, gibbs) ///
block({var_id2}, gibbs)   ///
mcmcsize(5000) dots
        


> (rsoshs <-  griphs  matu ntxhs       M1[id]) ///
> (griphs <- matu)

Fitting fixed-effects model:

Iteration 0:   log likelihood = -1041.2259  
Iteration 1:   log likelihood = -1041.2259  

Refining starting values:

Grid node 0:   log likelihood = -1031.7984

Fitting full model:

Iteration 0:   log likelihood = -1031.7984  (not concave)
Iteration 1:   log likelihood = -996.61688  
Iteration 2:   log likelihood = -991.28363  
Iteration 3:   log likelihood = -990.88709  
Iteration 4:   log likelihood = -990.88573  
Iteration 5:   log likelihood = -990.88573  

Generalized structural equation model           Number of obs     =        425

Response       : rsoshs
Family         : Gaussian
Link           : identity

Response       : griphs
Family         : Gaussian
Link           : identity

Log likelihood = -990.88573

 ( 1)  [rsoshs]M1[id] = 1
------------------------------------------------------------------------------
             |      Coef.   Std. Err.      z    P>|z|     [95% 

In [332]:
%%stata --data dm --openstata 

gsem ///
(rsoshs <-  griphs  matu ntxhs       M1[id]) ///
(griphs <- matu)


set seed 12345

bayesmh ///
(rsoshs griphs  matu ntxhs, reffects(id) likelihood(normal({var_0})) noconstant ) ///
(griphs matu, likelihood(normal({var_1})) ), ///
prior({rsoshs:i.id},  normal({rsoshs:_cons},{var_id})) ///
prior({rsoshs:_cons}, normal(0, 100)) ///
prior({rsoshs:griphs}, normal(0, 100)) ///
prior({rsoshs:matu}, normal(0, 100)) ///
prior({rsoshs:ntxhs}, normal(0, 100)) ///
///
prior({griphs:_cons}, normal(0, 100)) ///
prior({griphs:matu}, normal(0, 100)) ///
///
prior({var_0},        igamma(0.001, 0.001)) ///
prior({var_1},        igamma(0.001, 0.001)) ///
prior({var_id},       igamma(0.001, 0.001)) ///
block({rsoshs:_cons}, gibbs)  ///
block({var_0}, gibbs) block({var_id}, gibbs)  block({var_1}, gibbs)  ///
mcmcsize(5000) dots



> (rsoshs <-  griphs  matu ntxhs       M1[id]) ///
> (griphs <- matu)

Fitting fixed-effects model:

Iteration 0:   log likelihood = -1041.2259  
Iteration 1:   log likelihood = -1041.2259  

Refining starting values:

Grid node 0:   log likelihood = -1031.7984

Fitting full model:

Iteration 0:   log likelihood = -1031.7984  (not concave)
Iteration 1:   log likelihood = -996.61688  
Iteration 2:   log likelihood = -991.28363  
Iteration 3:   log likelihood = -990.88709  
Iteration 4:   log likelihood = -990.88573  
Iteration 5:   log likelihood = -990.88573  

Generalized structural equation model           Number of obs     =        425

Response       : rsoshs
Family         : Gaussian
Link           : identity

Response       : griphs
Family         : Gaussian
Link           : identity

Log likelihood = -990.88573

 ( 1)  [rsoshs]M1[id] = 1
------------------------------------------------------------------------------
             |      Coef.   Std. Err.      z    P>|z|     [95% 

In [335]:
%%stata --data dm --openstata 



set seed 12345

bayesmh///
(rsoshs i.id griphs, likelihood(normal({var_0})) noconstant) ///
(griphs i.id matu, likelihood(normal({var_1})) noconstant) ///
///
prior ({rsoshs:i.id},                                 ///
mvnormal(2, {rsoshs:_cons}, {griphs:griphs}, {covar,m}))    ///
///
block ({rsoshs: i.id},        reffects)                           ///
block ({griphs: i.id#c.week}, reffects)                           ///
///
prior({var_0},   igamma(0.01, 0.01)) block({var_0}, gibbs)        ///
prior({covar,m}, iwishart(2, 3, I(2))) block({covar,m}, gibbs)    ///
///
prior({rsoshs:griphs _cons}, normal(0, 1e2))                        ///
block({rsoshs:_cons}) block({rsoshs:griphs})
        


. > ///
> prior ({rsoshs:i.id i.id#c.griphs},                                 ///
> mvnormal(2, {rsoshs:_cons}, {rsoshs:griphs}, {covar,m}))    ///
> ///
> block ({rsoshs: i.id},        reffects)                           ///
> block ({rsoshs: i.id#c.week}, reffects)                           ///
> ///
> prior({var_0},   igamma(0.01, 0.01)) block({var_0}, gibbs)        ///
> prior({covar,m}, iwishart(2, 3, I(2))) block({covar,m}, gibbs)    ///
> ///
> prior({rsoshs:griphs _cons}, normal(0, 1e2))                        ///
> block({rsoshs:_cons}) block({rsoshs:griphs})
id#c.week: maxvar too small
    You have attempted to use an interaction with too many levels or attempted to fit a model with too many variables.  You need to increase maxvar; it is currently 5000.  Use set maxvar; see help 
    maxvar.

    If you are using factor variables and included an interaction that has lots of missing cells, either increase maxvar or set emptycells drop to reduce the required matrix size; see h

In [274]:
dm.head(10)

,id,session,sequence,seq,trips,nota,falk,rsos,tsos,grip,ptiso,pt60,godin,paq,ntxc,cint,calc,vitd,vitd2,mvh,matu,age,gender,griph,ptisoh,rsosh,tsosh,mvhh,ntxh,godinh,paqh,calch,cinth,rsoshs,tsoshs,griphs,ptisohs,ntxhs,mvhhs,godinhs,cinths
0,100,1,1,1,4.0,NaN,NaN,3828.0,3601.0,NaN,113.928055,69.291710,37.0,2.5866,711.848145,1572.000000,1077.668945,3.382,135.274002,105.709999,-1.67,11.75,0,23.770721,113.928055,3828.000000,3601.0,105.709999,711.848145,37.0,2.5866,1077.668945,1572.000000,0.106119,-0.813017,0.063802,-0.254023,0.681627,0.056194,-0.897923,-0.054358
1,100,1,3,2,4.0,NaN,NaN,3898.0,3629.0,27.0,136.023758,87.778656,48.0,3.1656,760.093933,2219.011963,1212.817993,1.025,41.018002,93.620003,-0.71,12.71,0,27.000000,136.023758,3898.000000,3629.0,93.620003,760.093933,48.0,3.1656,1212.817993,2219.011963,0.863356,-0.546909,0.544870,0.219285,0.870666,-0.271182,-0.619845,1.292007
2,100,1,5,3,4.0,1.0,NaN,3851.0,3677.0,37.0,177.053085,103.303604,44.0,3.2084,543.734619,2599.831055,874.338013,4.546,181.848007,98.139999,0.41,13.83,0,24.354441,177.053085,3822.112549,3677.0,98.139999,543.734619,44.0,3.2084,874.338013,2599.831055,0.042431,-0.090723,0.150759,1.098167,0.022918,-0.148788,-0.720964,2.084451
3,100,1,7,4,4.0,NaN,NaN,3952.0,3740.0,40.5,NaN,NaN,66.0,2.7597,454.364349,2482.117920,1672.296997,5.907,236.738998,88.139999,1.17,14.74,0,40.500000,134.567993,3952.000000,3740.0,88.139999,454.364349,66.0,2.7597,1672.296997,2482.117920,1.447509,0.508021,2.555976,0.188102,-0.327256,-0.419571,-0.164810,1.839503
4,101,1,1,1,1.0,NaN,NaN,3682.0,3603.0,NaN,133.048798,86.341110,25.0,1.7136,937.996765,1549.993042,1521.052002,7.432,297.299988,89.290001,-1.63,11.45,0,20.065380,133.048798,3682.000000,3603.0,89.290001,937.996765,25.0,1.7136,1521.052002,1549.993042,-1.473259,-0.794009,-0.488186,0.155559,1.567732,-0.388431,-1.201280,-0.100152
5,102,1,1,1,4.0,NaN,NaN,3935.0,3916.0,NaN,171.360291,83.164680,45.0,2.9459,435.431000,1308.953979,504.941986,3.238,129.501999,86.570000,-1.24,12.29,0,26.486280,171.360291,3935.000000,3916.0,86.570000,435.431000,45.0,2.9459,504.941986,1308.953979,1.263609,2.180701,0.468341,0.976223,-0.401441,-0.462084,-0.695685,-0.601729
6,102,1,3,2,4.0,NaN,NaN,3899.0,3863.0,27.0,NaN,NaN,102.0,3.8297,650.394531,1260.655029,771.072998,1.084,43.356998,68.709999,-0.27,13.26,0,27.000000,155.151871,3899.000000,3863.0,68.709999,650.394531,102.0,3.8297,771.072998,1260.655029,0.874173,1.676996,0.544870,0.629025,0.440837,-0.945701,0.745262,-0.702234
7,102,1,5,3,4.0,NaN,NaN,3878.0,3908.0,42.0,210.997086,134.422821,113.0,3.3557,906.064087,2509.722900,552.890015,1.895,89.841003,59.139999,0.77,14.30,0,42.000000,210.997086,3878.000000,3908.0,59.139999,906.064087,113.0,3.3557,552.890015,2509.722900,0.647002,2.104671,2.779432,1.825276,1.442612,-1.204840,1.023339,1.896946
8,102,1,7,4,4.0,NaN,NaN,3951.0,4015.0,42.0,NaN,NaN,123.0,2.4667,NaN,1685.947021,534.088013,3.337,135.897003,60.709999,2.08,15.38,0,42.000000,180.588852,3951.000000,4015.0,60.709999,402.240692,123.0,2.4667,534.088013,1685.947021,1.436692,3.121584,2.779432,1.173906,-0.531489,-1.162327,1.276137,0.182754
9,103,1,1,1,3.0,NaN,NaN,3768.0,3686.0,NaN,146.600281,90.861717,33.0,3.2554,737.074768,2551.882080,994.062988,3.764,150.544006,120.570000,-1.91,11.83,0,22.247978,146.600281,3768.000000,3686.0,120.570000,737.074768,33.0,3.2554,994.062988,2551.882080,-0.542940,-0.005188,-0.163043,0.445843,0.780471,0.458576,-0.999042,1.984675


In [275]:
dm.describe()

,id,session,sequence,seq,trips,nota,falk,rsos,tsos,grip,ptiso,pt60,godin,paq,ntxc,cint,calc,vitd,vitd2,mvh,matu,age,gender,griph,ptisoh,rsosh,tsosh,mvhh,ntxh,godinh,paqh,calch,cinth,rsoshs,tsoshs,griphs,ptisohs,ntxhs,mvhhs,godinhs,cinths
count,425.000000,425.000000,425.000000,425.000000,425.000000,50.0,11.0,415.000000,415.000000,379.000000,310.000000,310.000000,422.000000,425.000000,385.000000,422.000000,421.000000,421.000000,421.000000,307.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,4.250000e+02,4.250000e+02,4.250000e+02,4.250000e+02,4.250000e+02,4.250000e+02
mean,326.197647,1.430588,4.197647,1.868235,2.736471,1.0,1.0,3815.551807,3687.079518,23.438074,124.041290,84.050201,72.606635,3.144819,540.346619,1598.184814,976.547974,4.703625,189.981476,106.180466,-0.893577,11.850613,0.468235,23.342438,125.786736,3818.190186,3686.545898,103.634758,537.885498,72.519432,3.144819,976.546631,1598.122314,0.000003,-0.000001,-6.731819e-08,1.032212e-07,-1.234167e-07,-1.436121e-07,2.692727e-08,7.180606e-08
std,197.549787,0.495742,1.833518,0.877686,0.827711,0.0,0.0,100.909471,109.038357,7.364498,53.311306,33.349968,39.683328,0.662686,265.354980,482.215729,455.661133,3.152561,127.345634,41.705219,1.942006,2.003003,0.499578,6.712725,46.683552,92.441414,105.220322,36.930027,255.216660,39.557331,0.662686,453.506714,480.562256,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,100.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,3542.000000,3367.000000,9.000000,32.647167,28.224886,0.000000,1.250000,56.308327,469.496002,51.714001,0.000000,0.000000,13.570000,-5.300000,7.500000,0.000000,9.000000,32.647167,3542.000000,3381.000000,13.570000,56.308327,0.000000,1.250000,51.714001,469.496002,-2.987732,-2.903868,-2.136604e+00,-1.995126e+00,-1.886935e+00,-2.438795e+00,-1.833274e+00,-2.348554e+00
25%,138.000000,1.000000,3.000000,1.000000,2.000000,1.0,1.0,3746.500000,3619.000000,18.000000,84.335386,58.508888,44.000000,2.718600,369.744904,1263.682983,652.146973,2.649000,107.401001,78.139999,-2.290000,10.340000,0.000000,19.000000,92.666969,3767.000000,3623.699951,82.000000,380.352814,44.000000,2.718600,661.242004,1264.697021,-0.553758,-0.597280,-6.468964e-01,-7.094526e-01,-6.172508e-01,-5.858311e-01,-7.209645e-01,-6.938233e-01
50%,189.000000,1.000000,4.000000,2.000000,3.000000,1.0,1.0,3820.000000,3689.000000,22.250000,114.025639,76.993565,65.000000,3.197700,514.754761,1548.725037,915.741028,4.204000,173.792007,99.110001,-1.000000,11.850000,0.000000,22.654043,120.684532,3820.000000,3686.000000,98.931030,513.239929,65.000000,3.197700,920.513977,1549.993042,0.019578,-0.005188,-1.025507e-01,-1.092934e-01,-9.656724e-02,-1.273686e-01,-1.900895e-01,-1.001520e-01
75%,530.000000,2.000000,6.000000,2.000000,3.000000,1.0,1.0,3878.000000,3758.000000,27.500000,156.607567,102.830297,92.750000,3.663800,662.435608,1878.681793,1196.994019,6.464000,263.270996,133.389999,0.450000,13.230000,1.000000,26.486280,150.466110,3872.000000,3751.000000,119.959816,650.394531,92.000000,3.663800,1196.979004,1878.453003,0.582096,0.612563,4.683408e-01,5.286525e-01,4.408374e-01,4.420538e-01,4.924642e-01,5.833390e-01
max,583.000000,2.000000,7.000000,4.000000,4.000000,1.0,1.0,4140.000000,4015.000000,53.000000,343.836792,237.822296,234.000000,4.610400,1754.324463,3624.724121,2518.876953,21.211000,848.426025,235.000000,4.950000,17.549999,1.000000,53.000000,343.836792,4140.000000,4015.000000,235.000000,1754.324463,234.000000,4.610400,2518.876953,3624.724121,3.481230,3.121584,4.418110e+00,4.670811e+00,4.766299e+00,3.557139e+00,4.082191e+00,4.217147e+00


In [267]:
dm.columns

Index(['id', 'session', 'sequence', 'seq', 'trips', 'nota', 'falk', 'rsos',
       'tsos', 'grip', 'ptiso', 'pt60', 'godin', 'paq', 'ntxc', 'cint', 'calc',
       'vitd', 'vitd2', 'mvh', 'matu', 'age', 'gender', 'griph', 'ptisoh',
       'rsosh', 'tsosh', 'mvhh', 'ntxh', 'godinh', 'paqh', 'calch', 'cinth',
       'rsoshs', 'tsoshs', 'griphs', 'ptisohs', 'ntxhs', 'mvhhs', 'godinhs'],
      dtype='object')

In [272]:
%%stata --data dm --openstata 

constraint 1 M1[id]=M2[id]

gsem ///
(rsosh <-  griph  matu ntxh       M1[id]) ///
(tsosh <-  ptisoh matu ntxh mvhh  M2[id]) ///
(griph <-   matu godinh cinth) ///
(ptisoh <-  matu godinh cinth) , constraints(1)
matrix b = e(b)







> (rsosh <-  griph  matu ntxh mvhh  M1[id]) ///
> (tsosh <-  ptisoh matu ntxh mvhh  M2[id]) ///
> (griph <-   matu godinh cinth) ///
> (ptisoh <-  matu godinh cinth) ,constraints(1)
(note: constraint number 1 caused error r(111))

Fitting fixed-effects model:

Iteration 0:   log likelihood = -8396.1149  
Iteration 1:   log likelihood = -8396.1149  

Refining starting values:

Grid node 0:   log likelihood =   -8353.93

Fitting full model:

Iteration 0:   log likelihood =   -8353.93  (not concave)
Iteration 1:   log likelihood = -8314.3246  (not concave)
Iteration 2:   log likelihood = -8272.3864  (not concave)
Iteration 3:   log likelihood = -8258.4659  (not concave)
Iteration 4:   log likelihood = -8251.2939  (not concave)
Iteration 5:   log likelihood = -8249.0693  
Iteration 6:   log likelihood = -8247.9025  (not concave)
Iteration 7:   log likelihood = -8247.6797  
Iteration 8:   log likelihood = -8247.5023  (not concave)
Iteration 9:   log likelihood = -8247.3273  
Iteration 10: 

In [230]:
%%stata --graph -d dm

graph twoway (scatter tsosh mvhh) 

In [98]:
%%stata --data dm --openstata 

drop if trips==1

gsem ///
(rsosh <-  griph  matu ntxh M1[id]) ///
(tsosh <-  ptisoh matu mvhh M2[id]) ///
(griph <-   matu   ) ///
(ptisoh <-  matu   ) 
matrix b = e(b)

gsem ///
(rsosh <-  griph  matu  ntxh M1[id]) ///
(tsosh <-  ptisoh matu  mvhh M2[id]) ///
(griph <-   matu M3[id]) ///
(ptisoh <-  matu M4[id]) , from(b, skip)

gsem ///
(rsoshs <-  griphs  matu ntxhs M1[id]) ///
(tsoshs <-  ptisohs matu mvhhs M2[id]) ///
(griphs <-   matu   ) ///
(ptisohs <-  matu   ) 
matrix b = e(b)

gsem ///
(rsoshs <-  griphs  matu ntxhs M1[id]) ///
(tsoshs <-  ptisohs matu mvhhs M2[id]) ///
(griph <-   matu M3[id]) ///
(ptisoh <-  matu M4[id]) , from(b, skip)

'Failed to open Stata'

In [87]:
%%stata --data dm --openstata 

keep id seq trips rsosh tsosh griph ptisoh ntxh mvhh matu
drop if missing(rsosh)
drop if missing(tsosh)
drop if missing(griph)
drop if missing(ptisoh)
drop if missing(ntxh)
drop if missing(mvhh)
drop if missing(matu)

drop if trips==1
count

gsem ///
(rsosh <-  griph  matu ntxh M1[id]) ///
(tsosh <-  ptisoh matu mvhh M2[id]) ///
(griph <-   matu   ) ///
(ptisoh <-  matu   ) 
matrix b = e(b)

gsem ///
(rsosh <-  griph  matu  ntxh M1[id]) ///
(tsosh <-  ptisoh matu  mvhh M2[id]) ///
(griph <-   matu M3[id]) ///
(ptisoh <-  matu M4[id]) , from(b, skip)


. keep id rsosh tsosh griph ptisoh ntxh mvhh matu
(0 observations deleted)
(10 observations deleted)
(0 observations deleted)
(0 observations deleted)
(0 observations deleted)
(0 observations deleted)
(0 observations deleted)
  415
> (rsosh <-  griph  matu ntxh M1[id]) ///
> (tsosh <-  ptisoh matu mvhh M2[id]) ///
> (griph <-   matu   ) ///
> (ptisoh <-  matu   ) 

Fitting fixed-effects model:

Iteration 0:   log likelihood = -8172.5731  
Iteration 1:   log likelihood = -8172.5731  

Refining starting values:

Grid node 0:   log likelihood = -8142.0646

Fitting full model:

Iteration 0:   log likelihood = -8142.0646  (not concave)
Iteration 1:   log likelihood = -8086.9295  (not concave)
Iteration 2:   log likelihood = -8073.0673  (not concave)
Iteration 3:   log likelihood = -8054.6206  (not concave)
Iteration 4:   log likelihood = -8043.7581  
Iteration 5:   log likelihood = -8037.2353  (not concave)
Iteration 6:   log likelihood = -8035.9737  
Iteration 7:   log likelihood = -8035.

In [53]:
dm.describe()

,id,session,sequence,seq,trips,nota,falk,rsos,tsos,grip,...,mvh,matu,age,gender,griph,ptisoh,rsosh,tsosh,mvhh,ntxch
count,425.000000,425.000000,425.000000,425.000000,425.000000,50.0,11.0,415.000000,415.000000,379.000000,...,307.000000,425.000000,425.000000,425.000000,425.000000,415.000000,425.000000,415.000000,420.000000,420.000000
mean,326.197647,1.430588,4.197647,1.868235,2.736471,1.0,1.0,3815.551807,3687.079518,23.438074,...,106.180466,-0.893577,11.850613,0.468235,23.412334,131.787201,3815.268311,3687.079590,104.799309,538.985962
std,197.549787,0.495742,1.833518,0.877686,0.827711,0.0,0.0,100.909471,109.038357,7.364498,...,41.705219,1.942006,2.003003,0.499578,6.997608,34.685455,60.782661,71.749718,36.542171,255.051041
min,100.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,3542.000000,3367.000000,9.000000,...,13.570000,-5.300000,7.500000,0.000000,9.000000,72.146973,3554.000000,3367.000000,13.570000,56.308327
25%,138.000000,1.000000,3.000000,1.000000,2.000000,1.0,1.0,3746.500000,3619.000000,18.000000,...,78.139999,-2.290000,10.340000,0.000000,18.500000,114.988209,3780.298096,3646.712646,83.287502,384.042091
50%,189.000000,1.000000,4.000000,2.000000,3.000000,1.0,1.0,3820.000000,3689.000000,22.250000,...,99.110001,-1.000000,11.850000,0.000000,22.500000,126.535263,3814.430908,3676.513672,102.535000,521.473938
75%,530.000000,2.000000,6.000000,2.000000,3.000000,1.0,1.0,3878.000000,3758.000000,27.500000,...,133.389999,0.450000,13.230000,1.000000,27.000000,138.835381,3853.525146,3713.966064,118.456932,648.279327
max,583.000000,2.000000,7.000000,4.000000,4.000000,1.0,1.0,4140.000000,4015.000000,53.000000,...,235.000000,4.950000,17.549999,1.000000,53.000000,343.836792,3971.000000,4016.659912,235.000000,1754.324463


In [ ]:
%%stata --data dm --openstata

*sum rsosh tsosh griph ptisoh ntxh

tab trips
drop if trips==1

constraint 1 M1[id]=M2[id]

gsem ///
(rsos <-  griph  matu  M1[id]) ///
(tsos <-  ptisoh matu  M2[id]) ///
(griph <-   matu   ) ///
(ptisoh <-  matu   ) ,  constraints(1)
matrix b = e(b)


gsem ///
(rsos <-  griph  matu ntxh M1[id]) ///
(tsos <-  ptisoh matu mvhh M2[id]) ///
(griph <-   matu   ) ///
(ptisoh <-  matu   ) , from(b, skip)  constraints(1)
matrix b = e(b)



gsem ///
(rsos <-  griph  matu  ntxh M1[id]) ///
(tsos <-  ptisoh matu  mvhh M2[id]) ///
(griph <-   matu   M3[id]) ///
(ptisoh <-  matu   M4[id]) , from(b, skip) constraints(1)


In [61]:
%%stata --data dm --openstata

*sum rsosh tsosh griph ptisoh ntxh

tab trips
drop if trips==1

constraint 1 M1[id]=M2[id]

gsem ///
(rsos <-  griph  matu  M1[id]) ///
(tsos <-  ptisoh matu  M2[id]) ///
(griph <-   matu   ) ///
(ptisoh <-  matu   ) ,  constraints(1)
matrix b = e(b)


gsem ///
(rsos <-  griph  matu ntxh M1[id]) ///
(tsos <-  ptisoh matu mvhh M2[id]) ///
(griph <-   matu   ) ///
(ptisoh <-  matu   ) , from(b, skip)  constraints(1)
matrix b = e(b)



gsem ///
(rsos <-  griph  matu  ntxh M1[id]) ///
(tsos <-  ptisoh matu  mvhh M2[id]) ///
(griph <-   matu   M3[id]) ///
(ptisoh <-  matu   M4[id]) , from(b, skip) constraints(1)


'Failed to open Stata'

In [55]:
%%stata --data dm --openstata

*sum rsosh tsosh griph ptisoh ntxh

tab trips
drop if trips==1

gsem ///
(rsos <-  griph  matu  M1[id]) ///
(tsos <-  ptisoh matu  M1[id]) ///
(griph <-   matu   ) ///
(ptisoh <-  matu   ) 
matrix b = e(b)


gsem ///
(rsos <-  griph  matu  		M1[id]) ///
(tsos <-  ptisoh matu mvhh  M2[id]) ///
(griph <-   matu   ) ///
(ptisoh <-  matu   ) , from(b, skip)
matrix b = e(b)


gsem ///
(rsos <-  griph  matu  ntxh M1[id]) ///
(tsos <-  ptisoh matu  mvhh M2[id]) ///
(griph <-   matu   M3[id]) ///
(ptisoh <-  matu   M4[id]) , from(b, skip)



. gsem ///
> (rsos <-  griph  matu  M1[id]) ///
> (tsos <-  ptisoh matu  M1[id]) ///
> (griph <-   matu   ) ///
> (ptisoh <-  matu   ) 

Fitting fixed-effects model:

Iteration 0:   log likelihood = -8225.7985  
Iteration 1:   log likelihood = -8225.7985  

Refining starting values:

Grid node 0:   log likelihood = -8225.7739

Fitting full model:

Iteration 0:   log likelihood = -8225.7739  (not concave)
Iteration 1:   log likelihood =  -8190.627  (not concave)
Iteration 2:   log likelihood = -8152.3166  (not concave)
Iteration 3:   log likelihood = -8141.4184  
Iteration 4:   log likelihood = -8130.7432  
Iteration 5:   log likelihood = -8130.5141  (not concave)
Iteration 6:   log likelihood = -8126.7435  (not concave)
Iteration 7:   log likelihood = -8124.8415  
Iteration 8:   log likelihood = -8121.9066  
Iteration 9:   log likelihood =  -8115.588  
Iteration 10:  log likelihood = -8111.1424  (not concave)
Iteration 11:  log likelihood = -8109.8758  
Iteration 12:  log likelihood =

In [25]:
%%stata -d dm -gr

histogram ptisoh


(bin=20, start=32.647167, width=15.559481)
